TODO: I will add a requirements file for this stuff

In [ ]:
! pip install smartnoise-synth

In [ ]:
! pip install git+https://github.com/ryan112358/private-pgm.git


In [ ]:
! pip install diffprivlib

## Use PrivateDataGenerator to generate datasets for each publication

In [2]:
from private_data_generator import PrivateDataGenerator
from saw2018cross.saw2018cross import Saw2018Cross

publication = Saw2018Cross()
private_data_generator = PrivateDataGenerator(publication)
private_data_generator.generate()

/Users/lucasrosenblatt/opt/miniconda3/envs/heuristic_fairness/lib/python3.8/site-packages/mbi/__init__.py:15: UserWarning: MixtureInference disabled, please install jax and jaxlib
  warnings.warn('MixtureInference disabled, please install jax and jaxlib')


Index(['sex', 'race', 'SES', 'HigherSES', 'X1STU30OCC_STEM1',
       'X2STU30OCC_STEM1', 'ninth_grade_aspirations',
       'eleventh_grade_aspirations', 'stem_career_aspirations'],
      dtype='object')
Generating: saw2018cross
